In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn import svm
import pandas as pd
import numpy as np
import sys
import os

In [97]:
is_vae = False
# Just choose the name of the dataset directory
DATA_DIR = '/Users/tomas/Documents/FEUP/Tese/data/ml-20m/processed_70_10_20'
if is_vae:
    PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/vae')
else:
    PARSE_DATA_DIR = os.path.join(DATA_DIR, 'embeddings/cdae')
file = 'metadataset_k_20.csv'

In [98]:
#read in the data using pandas
metadataset = pd.read_csv(os.path.join(PARSE_DATA_DIR, file ))
metadataset.head()

,original_id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f41,f42,f43,f44,f45,f46,f47,f48,f49,first_place
0,1,2.092112e-32,6.249415e-32,-6.000611e-33,5.893090e-32,4.020546e-32,2.152657e-32,-5.836300e-32,2.339763e-32,4.832709e-32,...,1.450966e-33,-2.433296e-32,-1.128275e-33,-1.185880e-33,1.241357e-32,-6.068290e-33,3.252277e-32,2.221478e-32,9.671203e-34,zeroes
1,2,-1.855838e-32,-2.090192e-32,-1.898156e-32,2.504322e-32,-2.451386e-32,-6.685095e-34,-3.221332e-32,-4.198181e-32,-6.362588e-32,...,1.600629e-32,3.991929e-32,-1.531876e-32,-4.649628e-32,-4.278011e-32,2.160474e-32,-3.867348e-32,2.380331e-32,-2.335757e-32,als_ndcg
2,3,-1.169303e-32,-6.049711e-32,4.136449e-32,-9.663286e-33,-1.924273e-32,3.455974e-32,1.667681e-32,5.966751e-34,-2.579415e-32,...,5.994075e-33,-1.462464e-32,-1.664031e-32,2.628347e-33,-1.029011e-32,1.240914e-32,-1.170141e-32,-2.649629e-32,2.516531e-32,most_popular_ndcg
3,4,-2.586232e-32,-3.247477e-32,6.048788e-33,1.627761e-32,5.529978e-32,-1.838027e-33,1.555958e-32,5.057206e-32,-2.870304e-33,...,1.395353e-32,-2.553700e-32,1.501834e-32,4.594490e-32,1.732268e-32,1.482250e-32,-7.583220e-33,4.964787e-32,1.042384e-32,zeroes
4,5,-1.429949e-32,-1.369787e-32,-2.009418e-32,4.561260e-32,1.526659e-32,-2.487647e-32,5.173627e-33,-3.500297e-32,4.993383e-32,...,6.116732e-33,-1.546610e-32,-1.768405e-32,1.836809e-32,2.371812e-32,-1.883063e-32,5.819227e-32,4.135692e-32,-4.840399e-32,most_popular_ndcg


### Encoder

In [108]:
#als:0
#bpr:1
#lmf:2
#most_pop_3
#zeros:4
target_pre = metadataset['first_place'].values 
label_encoder = LabelEncoder()
target = label_encoder.fit_transform(target_pre)

### Normalization inputs

In [109]:
normalize = False

In [110]:
if normalize:
  #---- SET INPUTS -----
  scaler = StandardScaler()
  #Compute the mean and std to be used for later scaling.
  scaler.fit(metadataset.drop(columns=['first_place','original_id']))
  # Perform standardization by centering and scaling
  inputs_transform = scaler.transform(metadataset.drop(columns=['first_place','original_id']))
  inputs = pd.DataFrame(inputs_transform)
  inputs.head()
else:
  inputs = metadataset.drop(columns=['first_place','original_id'])

### Model

In [16]:
kf = KFold(n_splits=5)
kf.get_n_splits()
print(kf)

KFold(n_splits=5, random_state=None, shuffle=False)


In [129]:
params = {
    'C': 316.22776601683796, 
    'penalty': 'l2', 
    'solver': 'newton-cg',
    'random_state':0}

In [130]:
i = 1 
reports = []
for train_index, test_index in kf.split(inputs):
    print('iteration: ', i)
    #get data fold
    X_train, X_test = inputs.iloc[train_index], inputs.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    #start model 
    print('fit')
    clf = LogisticRegression(
        penalty=params['penalty'], 
        C=params['C'],
        solver=params['solver'],
        random_state=params['random_state'],
        verbose=5)
    
    clf.fit(X_train, y_train)
    print('predict')
    y_pred = clf.predict(X_test)
    
    report = classification_report(y_test, 
                                   y_pred, 
                                   target_names=np.unique(metadataset['first_place'].values),
                                  output_dict=True)
    reports.append(report)
    print('end: ', i)
    i+=1

iteration:  1
fit


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.8s finished
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn

predict
end:  1
iteration:  2
fit


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.6s finished
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


predict
end:  2
iteration:  3
fit


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.5s finished
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


predict
end:  4
iteration:  5
fit


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.8s remaining:    0.0s


predict
end:  5


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.4s finished
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [131]:
avg_reports = report_average(reports)
print_report(avg_reports)
#0.26368523962522367

+-------------------+---------------------+------------------------+-----------------------+
|     Algorithm     |      Precision      |         Recall         |           F1          |
+-------------------+---------------------+------------------------+-----------------------+
|      als_ndcg     |  0.2637136520921216 |   0.9995597287287195   |   0.4173148570509939  |
|      bpr_ndcg     |         0.0         |          0.0           |          0.0          |
|      lmf_ndcg     |         0.0         |          0.0           |          0.0          |
| most_popular_ndcg | 0.19864801864801865 | 0.0004129702916770963  | 0.0008241814064936374 |
|       zeroes      | 0.13714285714285715 | 0.00023536678365085475 | 0.0004698765415583628 |
|        ---        |         ---         |          ---           |          ---          |
|     macro avg     | 0.11990090557659946 |  0.20004161316080946   |  0.08372178299980917  |
|    weighted avg   | 0.13638207553330087 |   0.2637006526955073   |  

#### Prints function

In [23]:
def report_average(reports):
    mean_dict = dict()
    for label in reports[0].keys():
        dictionary = dict()

        if label in 'accuracy':
            mean_dict[label] = sum(d[label] for d in reports) / len(reports)
            continue

        for key in reports[0][label].keys():
            dictionary[key] = sum(d[label][key] for d in reports) / len(reports)
        mean_dict[label] = dictionary

    return mean_dict

In [93]:
def print_report(avg_reports):
    from prettytable import PrettyTable
    x = PrettyTable()

    x.field_names = ["Algorithm", "Precision", "Recall", "F1"]

    for label in avg_reports.keys():
        if label in 'accuracy':
            x.add_row(['---','---','---','---'])
            continue
        x.add_row([label, 
                   avg_reports[label]['precision'], 
                   avg_reports[label]['recall'], 
                   avg_reports[label]['f1-score']])


    print(x)
    print('Accuracy: ', avg_reports['accuracy'])

## Gridsearch

In [111]:
grid={
    "C":np.logspace(-5,5,9), 
    "penalty":["l2"],
    "solver":["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
}# l1 lasso l2 ridge

logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=5, verbose=5, n_jobs=-1)
logreg_cv.fit(inputs,target)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed: 15.4min finished
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': array([1.00000000e-05, 1.77827941e-04, 3.16227766e-03, 5.62341325e-02,
       1.00000000e+00, 1.77827941e+01, 3.16227766e+02, 5.62341325e+03,
       1.00000000e+05]),
                         'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                          

In [113]:
logreg_cv.best_params_

{'C': 316.22776601683796, 'penalty': 'l2', 'solver': 'newton-cg'}

In [104]:
logreg_cv.best_params_

{'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}

In [25]:
grid

{'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
 'penalty': ['l2'],
 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}